# Exploratory Data Analysis (EDA) - Used Cars Dataset

This notebook performs a comprehensive exploratory data analysis on a used cars dataset containing various vehicle characteristics and pricing information.

## Dataset Overview
The dataset contains information about used cars with the following variables:
- **brand**: Car brand/manufacturer
- **currency**: Price currency
- **description**: Vehicle description
- **fuel_type**: Type of fuel (Petrol, Diesel, etc.)
- **item_condition**: Condition of the vehicle
- **manufacturer**: Vehicle manufacturer
- **mileage_from_odometer**: Vehicle mileage
- **model_date**: Year of the model
- **price**: Vehicle price (target variable)
- **vehicle_engine**: Engine specifications
- **vehicle_transmission**: Transmission type


## 1. Data Loading and Initial Exploration

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Load the dataset
file = 'datasets/used_car_dataset.csv'
df = pd.read_csv(file)

print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")

## 2. Dataset Structure Analysis

In [ ]:
# Display basic information about the dataset
print("=== DATASET BASIC INFORMATION ===")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData Types:")
print(df.dtypes)
print(f"\nMemory Usage:")
print(df.info())

In [ ]:
# Display first few rows
print("=== FIRST 5 ROWS ===")
df.head()

In [ ]:
# Check for missing values
print("=== MISSING VALUES ANALYSIS ===")
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_data) > 0:
    print(missing_data)
    
    # Visualize missing values
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    missing_data.plot(x='Column', y='Missing_Count', kind='bar', ax=plt.gca())
    plt.title('Missing Values Count by Column')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    missing_data.plot(x='Column', y='Missing_Percentage', kind='bar', ax=plt.gca())
    plt.title('Missing Values Percentage by Column')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found in the dataset!")

## 3. Data Summary and Unique Values

In [ ]:
# Analyze categorical variables
categorical_cols = ['brand', 'currency', 'fuel_type', 'item_condition', 'manufacturer', 'vehicle_transmission']

print("=== CATEGORICAL VARIABLES SUMMARY ===")
for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col.upper()}:")
        print(f"Unique values: {df[col].nunique()}")
        print(f"Top 10 values:")
        print(df[col].value_counts().head(10))
        print("-" * 50)

In [ ]:
# Analyze numerical variables
numerical_cols = ['model_date', 'price']

print("=== NUMERICAL VARIABLES SUMMARY ===")
for col in numerical_cols:
    if col in df.columns:
        print(f"\n{col.upper()}:")
        print(df[col].describe())
        print("-" * 50)

# Overall statistical summary
print("\n=== OVERALL STATISTICAL SUMMARY ===")
df.describe(include='all')

## 4. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Remove the unnamed index column if it exists
if 'Unnamed: 0' in df_clean.columns:
    df_clean = df_clean.drop('Unnamed: 0', axis=1)

# Clean mileage data (remove 'km' and convert to numeric)
def clean_mileage(mileage_str):
    if pd.isna(mileage_str):
        return np.nan
    # Remove commas and 'km', then convert to int
    try:
        return int(str(mileage_str).replace(',', '').replace(' km', '').replace('km', ''))
    except:
        return np.nan

df_clean['mileage_numeric'] = df_clean['mileage_from_odometer'].apply(clean_mileage)

# Clean engine data (extract numeric value)
def clean_engine(engine_str):
    if pd.isna(engine_str):
        return np.nan
    try:
        # Extract numeric part from strings like '1490cc'
        numeric_part = ''.join(filter(str.isdigit, str(engine_str)))
        return int(numeric_part) if numeric_part else np.nan
    except:
        return np.nan

df_clean['engine_numeric'] = df_clean['vehicle_engine'].apply(clean_engine)

print("Data cleaning completed!")
print(f"Clean dataset shape: {df_clean.shape}")
print(f"\nNew numerical columns created:")
print(f"- mileage_numeric: {df_clean['mileage_numeric'].count()} non-null values")
print(f"- engine_numeric: {df_clean['engine_numeric'].count()} non-null values")

## 5. Univariate Analysis

### 5.1 Distribution of Numerical Variables

In [ ]:
# Price distribution analysis
plt.figure(figsize=(15, 10))

# Price histogram
plt.subplot(2, 3, 1)
plt.hist(df_clean['price'].dropna(), bins=50, alpha=0.7, color='skyblue')
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')

# Price log-scale histogram
plt.subplot(2, 3, 2)
price_positive = df_clean['price'][df_clean['price'] > 0].dropna()
plt.hist(np.log10(price_positive), bins=50, alpha=0.7, color='lightcoral')
plt.title('Price Distribution (Log Scale)')
plt.xlabel('Log10(Price)')
plt.ylabel('Frequency')

# Price box plot
plt.subplot(2, 3, 3)
plt.boxplot(df_clean['price'].dropna())
plt.title('Price Box Plot')
plt.ylabel('Price')

# Mileage distribution
plt.subplot(2, 3, 4)
plt.hist(df_clean['mileage_numeric'].dropna(), bins=50, alpha=0.7, color='lightgreen')
plt.title('Mileage Distribution')
plt.xlabel('Mileage (km)')
plt.ylabel('Frequency')

# Model year distribution
plt.subplot(2, 3, 5)
plt.hist(df_clean['model_date'].dropna(), bins=30, alpha=0.7, color='gold')
plt.title('Model Year Distribution')
plt.xlabel('Model Year')
plt.ylabel('Frequency')

# Engine size distribution
plt.subplot(2, 3, 6)
plt.hist(df_clean['engine_numeric'].dropna(), bins=50, alpha=0.7, color='plum')
plt.title('Engine Size Distribution')
plt.xlabel('Engine Size (cc)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Print summary statistics
print("=== NUMERICAL VARIABLES STATISTICS ===")
numerical_vars = ['price', 'mileage_numeric', 'model_date', 'engine_numeric']
for var in numerical_vars:
    if var in df_clean.columns:
        print(f"\n{var.upper()}:")
        print(df_clean[var].describe())

### 5.2 Frequency Analysis of Categorical Variables

In [ ]:
# Categorical variables visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Categorical Variables Distribution', fontsize=16)

# Brand distribution (top 10)
top_brands = df_clean['brand'].value_counts().head(10)
axes[0, 0].bar(range(len(top_brands)), top_brands.values, color='skyblue')
axes[0, 0].set_title('Top 10 Brands')
axes[0, 0].set_xticks(range(len(top_brands)))
axes[0, 0].set_xticklabels(top_brands.index, rotation=45)
axes[0, 0].set_ylabel('Frequency')

# Fuel type distribution
fuel_counts = df_clean['fuel_type'].value_counts()
axes[0, 1].pie(fuel_counts.values, labels=fuel_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('Fuel Type Distribution')

# Vehicle transmission distribution
trans_counts = df_clean['vehicle_transmission'].value_counts()
axes[0, 2].bar(trans_counts.index, trans_counts.values, color='lightcoral')
axes[0, 2].set_title('Transmission Type Distribution')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].tick_params(axis='x', rotation=45)

# Item condition distribution
condition_counts = df_clean['item_condition'].value_counts()
axes[1, 0].bar(condition_counts.index, condition_counts.values, color='lightgreen')
axes[1, 0].set_title('Item Condition Distribution')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].tick_params(axis='x', rotation=45)

# Currency distribution
currency_counts = df_clean['currency'].value_counts()
axes[1, 1].pie(currency_counts.values, labels=currency_counts.index, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Currency Distribution')

# Top manufacturers
top_manufacturers = df_clean['manufacturer'].value_counts().head(10)
axes[1, 2].bar(range(len(top_manufacturers)), top_manufacturers.values, color='gold')
axes[1, 2].set_title('Top 10 Manufacturers')
axes[1, 2].set_xticks(range(len(top_manufacturers)))
axes[1, 2].set_xticklabels(top_manufacturers.index, rotation=45)
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 6. Bivariate Analysis

### 6.1 Price Relationships with Other Variables

In [ ]:
# Price vs numerical variables
plt.figure(figsize=(15, 10))

# Price vs Mileage
plt.subplot(2, 2, 1)
# Sample data to avoid overplotting
sample_data = df_clean.dropna(subset=['price', 'mileage_numeric']).sample(n=min(5000, len(df_clean)))
plt.scatter(sample_data['mileage_numeric'], sample_data['price'], alpha=0.5, color='blue')
plt.xlabel('Mileage (km)')
plt.ylabel('Price')
plt.title('Price vs Mileage')

# Price vs Model Year
plt.subplot(2, 2, 2)
sample_data2 = df_clean.dropna(subset=['price', 'model_date']).sample(n=min(5000, len(df_clean)))
plt.scatter(sample_data2['model_date'], sample_data2['price'], alpha=0.5, color='green')
plt.xlabel('Model Year')
plt.ylabel('Price')
plt.title('Price vs Model Year')

# Price vs Engine Size
plt.subplot(2, 2, 3)
sample_data3 = df_clean.dropna(subset=['price', 'engine_numeric']).sample(n=min(5000, len(df_clean)))
plt.scatter(sample_data3['engine_numeric'], sample_data3['price'], alpha=0.5, color='red')
plt.xlabel('Engine Size (cc)')
plt.ylabel('Price')
plt.title('Price vs Engine Size')

# Price distribution by fuel type
plt.subplot(2, 2, 4)
fuel_types = df_clean['fuel_type'].value_counts().head(5).index
price_by_fuel = [df_clean[df_clean['fuel_type'] == fuel]['price'].dropna() for fuel in fuel_types]
plt.boxplot(price_by_fuel, labels=fuel_types)
plt.xlabel('Fuel Type')
plt.ylabel('Price')
plt.title('Price Distribution by Fuel Type')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Price analysis by categorical variables
plt.figure(figsize=(18, 12))

# Price by Brand (top 10)
plt.subplot(2, 3, 1)
top_brands = df_clean['brand'].value_counts().head(10).index
price_by_brand = [df_clean[df_clean['brand'] == brand]['price'].dropna() for brand in top_brands]
plt.boxplot(price_by_brand, labels=top_brands)
plt.xlabel('Brand')
plt.ylabel('Price')
plt.title('Price Distribution by Brand (Top 10)')
plt.xticks(rotation=45)

# Price by Transmission
plt.subplot(2, 3, 2)
transmission_types = df_clean['vehicle_transmission'].dropna().unique()
price_by_trans = [df_clean[df_clean['vehicle_transmission'] == trans]['price'].dropna() for trans in transmission_types]
plt.boxplot(price_by_trans, labels=transmission_types)
plt.xlabel('Transmission')
plt.ylabel('Price')
plt.title('Price Distribution by Transmission')

# Price by Condition
plt.subplot(2, 3, 3)
conditions = df_clean['item_condition'].dropna().unique()
price_by_condition = [df_clean[df_clean['item_condition'] == cond]['price'].dropna() for cond in conditions]
plt.boxplot(price_by_condition, labels=conditions)
plt.xlabel('Condition')
plt.ylabel('Price')
plt.title('Price Distribution by Condition')
plt.xticks(rotation=45)

# Average price by brand (top 10)
plt.subplot(2, 3, 4)
avg_price_brand = df_clean.groupby('brand')['price'].mean().sort_values(ascending=False).head(10)
plt.bar(range(len(avg_price_brand)), avg_price_brand.values, color='lightblue')
plt.xlabel('Brand')
plt.ylabel('Average Price')
plt.title('Average Price by Brand (Top 10)')
plt.xticks(range(len(avg_price_brand)), avg_price_brand.index, rotation=45)

# Average price by fuel type
plt.subplot(2, 3, 5)
avg_price_fuel = df_clean.groupby('fuel_type')['price'].mean().sort_values(ascending=False)
plt.bar(avg_price_fuel.index, avg_price_fuel.values, color='lightcoral')
plt.xlabel('Fuel Type')
plt.ylabel('Average Price')
plt.title('Average Price by Fuel Type')
plt.xticks(rotation=45)

# Price count by year
plt.subplot(2, 3, 6)
df_clean['model_date'] = pd.to_numeric(df_clean['model_date'], errors='coerce')
year_counts = df_clean['model_date'].value_counts().sort_index()
plt.plot(year_counts.index, year_counts.values, marker='o', linestyle='-', color='green')
plt.xlabel('Model Year')
plt.ylabel('Number of Cars')
plt.title('Number of Cars by Model Year')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### 6.2 Correlation Analysis

In [ ]:
# Correlation analysis for numerical variables
numerical_vars = ['price', 'mileage_numeric', 'model_date', 'engine_numeric']
correlation_data = df_clean[numerical_vars].corr()

# Create correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix - Numerical Variables')
plt.tight_layout()
plt.show()

print("=== CORRELATION MATRIX ===")
print(correlation_data)

# Find strongest correlations with price
price_correlations = correlation_data['price'].sort_values(key=abs, ascending=False)
print("\n=== CORRELATIONS WITH PRICE (sorted by absolute value) ===")
for var, corr in price_correlations.items():
    if var != 'price':
        print(f"{var}: {corr:.3f}")

## 7. Key Insights and Findings

In [ ]:
# Generate comprehensive summary insights
print("=== KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS ===")
print()

# Dataset overview
print("1. DATASET OVERVIEW:")
print(f"   • Total records: {len(df_clean):,}")
print(f"   • Total features: {df_clean.shape[1]}")
print(f"   • Missing values: {df_clean.isnull().sum().sum()}")
print()

# Price insights
print("2. PRICE ANALYSIS:")
price_stats = df_clean['price'].describe()
print(f"   • Average price: ${price_stats['mean']:,.2f}")
print(f"   • Median price: ${price_stats['50%']:,.2f}")
print(f"   • Price range: ${price_stats['min']:,.2f} - ${price_stats['max']:,.2f}")
print(f"   • Standard deviation: ${price_stats['std']:,.2f}")
print()

# Brand insights
print("3. BRAND ANALYSIS:")
top_brand = df_clean['brand'].value_counts().index[0]
top_brand_count = df_clean['brand'].value_counts().iloc[0]
total_brands = df_clean['brand'].nunique()
print(f"   • Most common brand: {top_brand} ({top_brand_count:,} cars, {top_brand_count/len(df_clean)*100:.1f}%)")
print(f"   • Total number of brands: {total_brands}")
avg_price_by_brand = df_clean.groupby('brand')['price'].mean().sort_values(ascending=False)
highest_avg_brand = avg_price_by_brand.index[0]
highest_avg_price = avg_price_by_brand.iloc[0]
print(f"   • Highest average price brand: {highest_avg_brand} (${highest_avg_price:,.2f})")
print()

# Fuel type insights
print("4. FUEL TYPE ANALYSIS:")
fuel_counts = df_clean['fuel_type'].value_counts()
most_common_fuel = fuel_counts.index[0]
print(f"   • Most common fuel type: {most_common_fuel} ({fuel_counts.iloc[0]:,} cars, {fuel_counts.iloc[0]/len(df_clean)*100:.1f}%)")
avg_price_by_fuel = df_clean.groupby('fuel_type')['price'].mean().sort_values(ascending=False)
print(f"   • Fuel types by average price:")
for fuel, price in avg_price_by_fuel.head(3).items():
    print(f"     - {fuel}: ${price:,.2f}")
print()

# Model year insights
print("5. MODEL YEAR ANALYSIS:")
year_stats = df_clean['model_date'].describe()
print(f"   • Year range: {year_stats['min']:.0f} - {year_stats['max']:.0f}")
print(f"   • Average model year: {year_stats['mean']:.1f}")
most_common_year = df_clean['model_date'].mode().iloc[0] if not df_clean['model_date'].mode().empty else 'N/A'
print(f"   • Most common model year: {most_common_year:.0f}")
print()

# Transmission insights
print("6. TRANSMISSION ANALYSIS:")
trans_counts = df_clean['vehicle_transmission'].value_counts()
print(f"   • Most common transmission: {trans_counts.index[0]} ({trans_counts.iloc[0]:,} cars, {trans_counts.iloc[0]/len(df_clean)*100:.1f}%)")
avg_price_by_trans = df_clean.groupby('vehicle_transmission')['price'].mean().sort_values(ascending=False)
print(f"   • Average prices by transmission:")
for trans, price in avg_price_by_trans.items():
    print(f"     - {trans}: ${price:,.2f}")
print()

# Correlation insights
print("7. CORRELATION INSIGHTS:")
price_corr = df_clean[['price', 'mileage_numeric', 'model_date', 'engine_numeric']].corr()['price']
price_corr = price_corr.drop('price').sort_values(key=abs, ascending=False)
print("   • Strongest correlations with price:")
for var, corr in price_corr.items():
    direction = "positive" if corr > 0 else "negative"
    strength = "strong" if abs(corr) > 0.5 else "moderate" if abs(corr) > 0.3 else "weak"
    print(f"     - {var}: {corr:.3f} ({strength} {direction} correlation)")
print()

print("8. ADDITIONAL OBSERVATIONS:")
print(f"   • Dataset appears to be primarily from Pakistan (PKR currency dominant)")
print(f"   • Price distribution is right-skewed with some high-value outliers")
print(f"   • Mileage data requires cleaning (contains text format)")
print(f"   • Engine size data also requires cleaning (contains 'cc' units)")
print(f"   • Missing values are minimal, making the dataset quite complete")
print()

print("=== END OF ANALYSIS ===")